In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install transformers
!pip install -U bitsandbytes
!pip install 'accelerate>=0.20.0'
!pip install datasets
!pip install peft
!pip install einops
!pip install trl
!pip install gpu-utils
!pip install GPUtil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 6.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 8.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for colored: filename=colored-1.4.4-py3-none-any.whl size=14247 sha256=ef58616f6afa4deda9d4c71d5e1bf33809d53d6f34564694f3595ddf4587fe38
  Stored in directory: /root/.cache/pip/wheels/e7/c3/07/fabb0941ff5df7a487d43a67081273045536cc96d4d0e816b4
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19172 sha256=5e3cf5f86131876a18e8e1efe7a7f004c0d69898c4e7d5bbd00ef3a3a322c9bf
  Stored in directory: /root/.cache/pip/wheels/5c/d8/c0/46899f8be7a7

In [3]:
import torch
from kaggle_secrets import UserSecretsClient
import transformers
import warnings

warnings.filterwarnings('ignore')

In [4]:
from datasets import load_dataset
snli=load_dataset('snli')
train_dataset=snli['train']
val_dataset=snli['validation']
test_dataset=snli['test']
train_dataset = train_dataset.select(range(0, len(train_dataset), 550))
val_dataset=val_dataset.select(range(0,len(val_dataset),100))
test_dataset=test_dataset.select(range(0,len(test_dataset),100))


README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/412k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/413k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/19.6M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/550152 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", quantization_config=bnb_config, device_map={"": 0})
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2",padding_side="left",add_eos_token=True,add_fast=False,device_map={"": 0})
if tokenizer.pad_token_id is None or tokenizer.pad_token_id == tokenizer.eos_token_id:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [9]:
import re
def construct_prompt(premise,hypothesis):
    return f"Given the premise {premise} and the hypothesis {hypothesis}, determine whether the hypothesis entails the premise. Output 0 in case of entailment, 1 in case the statements are unrealted or not directly implied, 2 in case of a contradiction. Please answer with only the number: 0, 1, or 2."

def extract_label(response):
    
    match = re.search(r'##\s*OUTPUT\s*([012])', response)
    if match:
        return int(match.group(1))
    
    match = re.findall(r'\b([012])\b', response)
    if match:
        return int(match[-1])  

    return None

def getAccuracy(predicted_labels,true_labels):
    no_correct=0
    total=len(predicted_labels)
    for i in range(total):
        if(predicted_labels[i]==true_labels[i]):
            no_correct+=1
    return (no_correct/total)*100
generation_args = {
    'max_new_tokens': 50,
    'temperature': 0.7,
    'pad_token_id': tokenizer.eos_token_id
}
  
Labels=[]
predicted_Labels=[]
for i in test_dataset:
    prompt = construct_prompt(i['premise'], i['hypothesis'])
    label = i['label']
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    attention_mask = inputs['attention_mask']
    with torch.no_grad():
        generated_ids = generated_ids = model.generate(inputs['input_ids'], attention_mask=attention_mask, **generation_args)
        response = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    predicted_label = extract_label(response)
    if predicted_label is not None:
        predicted_Labels.append(predicted_label)
    else:
        print(f"Warning: Could not extract label from response '{response}'")
    if(predicted_label!=label):
        print(response)
        print(predicted_label,label)
    Labels.append(label)

accuracy = getAccuracy(predicted_Labels, Labels)
print(f"Accuracy on the model before fine-tuning is {accuracy}")

Given the premise This church choir sings to the masses as they sing joyous songs from the book at a church. and the hypothesis The church has cracks in the ceiling., determine whether the hypothesis entails the premise. Output 0 in case of entailment, 1 in case the statements are unrealted or not directly implied, 2 in case of a contradiction. Please answer with only the number: 0, 1, or 2.
## INPUT

##OUTPUT
2

2 1
Given the premise A woman within an orchestra is playing a violin. and the hypothesis A woman is playing the violin., determine whether the hypothesis entails the premise. Output 0 in case of entailment, 1 in case the statements are unrealted or not directly implied, 2 in case of a contradiction. Please answer with only the number: 0, 1, or 2.

```python
# Solution
premise = "A woman within an orchestra is playing a violin."
hypothesis = "A woman is playing the violin."

# Check if the hypothesis entails the premise
if "woman
2 0
Given the premise many children play in the

In [ ]:
from functools import partial
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.config.pad_token_id = tokenizer.pad_token_id


def construct_prompt_for_finetune(premise,hypothesis,label):
    Intro="Below is the demonstration of a Natural Language Inference task "
    Instruct=f"### Instruct: Given the premise {premise} and the hypothesis {hypothesis}, find whether the hypothesis entails the premise. Output 0 for entailment, 1 for neutral and 2 for contradiction "
    Response=f"### Response: Answer: {label} "
    End="### End"
    return Intro+Instruct+Response+End


def preprocess_for_finetune(example):
    prompt = construct_prompt_for_finetune(example['premise'], example['hypothesis'], example['label'])
    return {'input_text': prompt}

def get_max_length(model):
    max_length = 512  
    print(f"Using max length: {max_length}")
    return max_length
def preprocess_batch(batch, tokenizer, max_length):
    inputs = tokenizer(
        batch["input_text"],
        max_length=max_length,
        padding="max_length",  
        truncation=True, 
        return_tensors='pt'
    )
    
    inputs["input_ids"] = inputs["input_ids"].squeeze() if inputs["input_ids"].dim() > 2 else inputs["input_ids"]
    inputs["attention_mask"] = inputs["attention_mask"].squeeze() if inputs["attention_mask"].dim() > 2 else inputs["attention_mask"]
    return inputs

def preprocess_dataset(tokenizer, model, dataset, seed):
    max_length = get_max_length(model)

    
    _preprocessing_function = partial(preprocess_batch, tokenizer=tokenizer, max_length=max_length)
    
    dataset = dataset.map(preprocess_for_finetune)
    dataset = dataset.map(_preprocessing_function)

    
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) <= max_length)

    dataset = dataset.shuffle(seed=seed)
    
    return dataset

seed = 42  
preprocessed_train_dataset = preprocess_dataset(tokenizer, model, train_dataset, seed)
preprocessed_val_dataset = preprocess_dataset(tokenizer, model, val_dataset, seed)


Using max length: 512


Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1001 [00:00<?, ? examples/s]

Using max length: 512


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
preprocessed_train_dataset = preprocessed_train_dataset.remove_columns(['premise', 'hypothesis', 'label'])
preprocessed_val_dataset = preprocessed_val_dataset.remove_columns(['premise', 'hypothesis', 'label'])

print(preprocessed_train_dataset.column_names)

#preprocessed_val_dataset = preprocessed_val_dataset.filter(lambda sample: sample['labels'] != -1)

['input_text', 'input_ids', 'attention_mask']


In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training,helpers,PeftModel

model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=32, 
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    lora_dropout=0.05,  
    task_type="CAUSAL_LM",
)

model.gradient_checkpointing_enable()
model.config.use_cache = False
model.config.pretraining_tp = 1
model.config.output_hidden_states = False

In [13]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [14]:
gpu_load_total = 0
memory_used_total = 0
samples_count = 0

def log_gpu_utilization():
    global gpu_load_total, memory_used_total, samples_count
    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        gpu_load_total += gpu.load * 100
        memory_used_total += gpu.memoryUsed
    samples_count += 1

def get_average_gpu_usage():
    if samples_count == 0:
        return 0, 0  # Avoid division by zero
    avg_gpu_load = gpu_load_total / samples_count
    avg_memory_used = memory_used_total / samples_count
    return avg_gpu_load, avg_memory_used

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer,SFTConfig
import time
import GPUtil
output_dir = f'./phi-nli-finetuned-{str(int(time.time()))}'
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=False,
    fp16=True,
    gradient_checkpointing=True,
    max_grad_norm=1.0
)

data_collator = transformers.DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)
class CustomSFTTrainer(SFTTrainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        inputs['input_ids']=inputs['input_ids'].squeeze(1)
        inputs['attention_mask'] = inputs['attention_mask'].squeeze(1)
        labels = inputs['input_ids'].clone()
        labels[labels == tokenizer.pad_token_id] = -100  
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
        loss = outputs.loss
        log_gpu_utilization()
        return (loss, outputs) if return_outputs else loss

    def on_epoch_end(self):
        self.epoch_number += 1
    
        epoch_dir = os.path.join(self.args.output_dir, f"epoch_{self.epoch_number}")
        os.makedirs(epoch_dir, exist_ok=True)
        self.save_model(epoch_dir)
        print(f"Model saved to {epoch_dir} after epoch {self.epoch_number}")

trainer = CustomSFTTrainer(
    model=model,
    peft_config=peft_config,
    args=training_args,
    train_dataset=preprocessed_train_dataset,
    eval_dataset=preprocessed_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

peft_model = get_peft_model(model, peft_config)
PeftModel.print_trainable_parameters(peft_model)
start=time.time()
trainer.train()
end=time.time()
total_time=end-start
hours, remainder = divmod(total_time, 3600)
minutes, seconds = divmod(remainder, 60)
print(f"Total time taken {hours} hours, {minutes} minutes and {seconds} seconds")
# trainer.save_model('./fine_tuned_model')

trainable params: 20,971,520 || all params: 2,800,655,360 || trainable%: 0.7488


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.752500,0.896451
2,0.896400,0.875910
3,0.812300,0.877453
4,0.639700,0.873733
5,0.728000,0.873113


Total time taken 2.0 hours, 30.0 minutes and 33.17526078224182 seconds


In [16]:
avg_load,avg_memory=get_average_gpu_usage()
print(f"Training used up a average load of {avg_load} and a average memory of {avg_memory} over the 5 epochs")

Training used up a average load of 99.99981834695731 and a average memory of 4052.873569482289 over the 5 epochs


In [ ]:
def extract_label(response):
    match = re.search(r'(answer|response):\s*([012])\b', response, re.IGNORECASE)
    if match:
        return int(match.group(2))
    return None
print(extract_label("Given the premise This church choir sings to the masses as they sing joyous songs from the book at a church. and the hypothesis The church has cracks in the ceiling., determine whether the hypothesis entails the premise. Output 0 in case of entailment, 1 in case the statements are unrealted or not directly implied, 2 in case of a contradiction. Please answer with only the number: 0, 1, or 2. Answer: 2 The hypothesis entails the premise.Answer: 2Explanation: The hypothesis states that the church has cracks in the ceiling. The premise states that this church choir sings to the masses as they sing joyous songs from the"))

2


In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(model, "/kaggle/working/phi-nli-finetuned-1730221175/checkpoint-4004", torch_dtype=torch.float16, is_trainable=False, device_map='cuda')
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id or tokenizer.add_special_tokens({'pad_token': '[PAD]'})
Labels = []
predicted_Labels = []

for i in test_dataset:
    prompt = construct_prompt(i['premise'], i['hypothesis'])
    label = i['label']
    
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True)
    inputs = {k: v.to(ft_model.device) for k, v in inputs.items()}  
    with torch.no_grad():
        generated_ids = ft_model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_new_tokens=50,pad_token_id=tokenizer.pad_token_id)
        response = tokenizer.decode(generated_ids[0], skip_special_tokens=True).strip()
    predicted_label = extract_label(response)
    
    if predicted_label is not None:
        predicted_Labels.append(predicted_label)
    else:
        print(f"Warning: Could not extract label from response '{response}'")
    if(predicted_label!=label):
        print(response)
        print(predicted_label,label)
    Labels.append(label)
accuracy = getAccuracy(predicted_Labels, Labels)
print(f"Accuracy on the model after fine-tuning is {accuracy}")


Given the premise This church choir sings to the masses as they sing joyous songs from the book at a church. and the hypothesis The church has cracks in the ceiling., determine whether the hypothesis entails the premise. Output 0 in case of entailment, 1 in case the statements are unrealted or not directly implied, 2 in case of a contradiction. Please answer with only the number: 0, 1, or 2. Answer: 2
The hypothesis entails the premise.
Answer: 2
Explanation: The hypothesis states that the church has cracks in the ceiling. The premise states that this church choir sings to the masses as they sing joyous songs from the
2 1
Given the premise Two men climbing on a wooden scaffold. and the hypothesis Two sad men climbing on a wooden scaffold., determine whether the hypothesis entails the premise. Output 0 in case of entailment, 1 in case the statements are unrealted or not directly implied, 2 in case of a contradiction. Please answer with only the number: 0, 1, or 2. Answer: 2
The hypothes

In [ ]:
save_directory = "/kaggle/working/main_model"

ft_model.save_pretrained(save_directory)

tokenizer.save_pretrained(save_directory)

print(f"Model saved to {save_directory}")